In [4]:
import cv2
import numpy as np
import math
import time
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
from keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from gtts import gTTS
import pygame
import os
import uuid
from threading import Thread

# Initialize hand detection
cap = cv2.VideoCapture(0)
hand_detector = HandDetector(maxHands=1)
hand_classifier = Classifier(r"C:\Users\ihebm\Desktop\CleanEmotion\models\mod1\signG4.h5", r"C:\Users\ihebm\Desktop\CleanEmotion\models\mod1\labels.txt")
hand_labels = ["Hello", "Bye", "Okay", "Thanks"]
offset = 20
imgSize = 300

# Initialize emotion detection
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
emotion_classifier = load_model(r'final_emotion_modelG1.h5')
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

emotion_start_time = time.time()
hand_sign_start_time = time.time()
last_dominant_emotion = None
last_hand_sign = None

# Initialize pygame for audio
pygame.mixer.init()

def draw_progress_bar(img, emotion, value, x, y, width=200, height=20):
    cv2.rectangle(img, (x, y), (x + width, y + height), (255, 255, 255), 1)
    filled_width = int(width * value)
    cv2.rectangle(img, (x, y), (x + filled_width, y + height), (0, 255, 0), -1)
    
    text = f"{emotion}: {value:.2f}"
    (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    text_x = x + (width - text_width) // 2
    text_y = y + height - (height - text_height) // 2
    
    cv2.putText(img, text, (text_x - 1, text_y - 1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

def speak(text):
    filename = f"response_{uuid.uuid4()}.mp3"
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)
    pygame.mixer.music.unload()
    time.sleep(0.5)
    try:
        os.remove(filename)
    except PermissionError:
        print(f"Could not delete {filename}, will try again later")

def respond_to_emotion(emotion):
    if emotion == 'Happy':
        speak("Good to see you happy!")
    elif emotion == 'Sad':
        speak("Feeling down? Here's a joke for you: Why don't scientists trust atoms? Because they make up everything!")
    elif emotion == 'Angry':
        speak("Take a deep breath. Let's stay calm.")
    elif emotion == 'Surprise':
        speak("Wow! That's surprising!")
    elif emotion == 'Fear':
        speak("Don't worry, everything will be alright.")
    elif emotion == 'Disgust':
        speak("That doesn't look pleasant. Let's focus on something nice.")
    else:
        speak("How can I assist you today?")

def respond_to_hand_sign(sign):
    if sign == "Hello":
        speak("Hello, Ehab! How are you?")
    elif sign == "Bye":
        speak("Goodbye, Ehab! Have a great day!")
    elif sign == "Okay":
        speak("Glad to see everything is okay!")
    elif sign == "Thanks":
        speak("You're welcome, Ehab!")

def process_frame(frame):
    global last_dominant_emotion, last_hand_sign, emotion_start_time, hand_sign_start_time

    imgOutput = frame.copy()
    
    # Hand sign detection
    hands, _ = hand_detector.findHands(frame)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = frame[y-offset:y + h + offset, x-offset:x + w + offset]
        if imgCrop.size > 0:
            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
            
            prediction, index = hand_classifier.getPrediction(imgWhite, draw=False)
            cv2.rectangle(imgOutput, (x-offset, y-offset-70), (x-offset+400, y-offset+60-50), (0, 255, 0), cv2.FILLED)
            cv2.putText(imgOutput, hand_labels[index], (x, y-30), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2)
            cv2.rectangle(imgOutput, (x-offset, y-offset), (x + w + offset, y + h + offset), (0, 255, 0), 4)
            
            # Respond to hand sign after 5 seconds
            if hand_labels[index] != last_hand_sign:
                hand_sign_start_time = time.time()
                last_hand_sign = hand_labels[index]
            elif time.time() - hand_sign_start_time > 5:
                Thread(target=respond_to_hand_sign, args=(hand_labels[index],)).start()
                hand_sign_start_time = time.time()

    # Emotion detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    for (x,y,w,h) in faces:
        cv2.rectangle(imgOutput, (x,y), (x+w,y+h), (0,255,255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48), interpolation=cv2.INTER_AREA)
        
        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            prediction = emotion_classifier.predict(roi)[0]
            label = emotion_labels[prediction.argmax()]
            label_position = (x, y)
            cv2.putText(imgOutput, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            
            # Draw progress bars
            for i, (emotion, value) in enumerate(zip(emotion_labels, prediction)):
                draw_progress_bar(imgOutput, emotion, value, 10, 50 + i * 25)
            
            # Update dominant emotion
            dominant_emotion = emotion_labels[prediction.argmax()]
            if dominant_emotion != last_dominant_emotion:
                emotion_start_time = time.time()
                last_dominant_emotion = dominant_emotion
            elif time.time() - emotion_start_time > 10:
                Thread(target=respond_to_emotion, args=(dominant_emotion,)).start()
                emotion_start_time = time.time()
            
            # Draw emotion indicator circle
            circle_x = 30
            circle_y = imgOutput.shape[0] - 30
            circle_color = (255, 255, 255)  # Default white
            circle_radius = 10  # Initial size
            emotion_duration = time.time() - emotion_start_time
            if emotion_duration > 5:
                circle_radius = 15  # Scale up after 5 seconds
                if dominant_emotion == 'Happy':
                    circle_color = (0, 255, 0)  # Green
                elif dominant_emotion == 'Sad':
                    circle_color = (0, 165, 255)  # Orange
                elif dominant_emotion in ['Fear', 'Angry']:
                    circle_color = (0, 0, 255)  # Red
                elif dominant_emotion in ['Surprise', 'Disgust']:
                    circle_color = (0, 255, 255)  # Yellow
            if emotion_duration > 60 and dominant_emotion in ['Sad', 'Neutral']:
                circle_color = (255, 0, 0)  # Blue
                cv2.putText(imgOutput, "Autism risk", (circle_x - 30, circle_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.circle(imgOutput, (circle_x, circle_y), circle_radius, circle_color, -1)
        else:
            cv2.putText(imgOutput, 'No Faces', (30,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    return imgOutput

def cleanup_audio_files():
    for file in os.listdir():
        if file.endswith(".mp3") and file.startswith("response"):
            try:
                os.remove(file)
            except PermissionError:
                print(f"Could not delete {file}")

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    output_frame = process_frame(frame)
    cv2.imshow('Hand Sign and Emotion Detector', output_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cleanup_audio_files()
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step


Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connection.py", line 196, in _new_conn
    sock = connection.create_connection(
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connectionpool.py", line 490, in _m

1/1 [==============================] - 0s 17ms/step


Exception in thread Thread-12:
Traceback (most recent call last):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connection.py", line 196, in _new_conn
    sock = connection.create_connection(
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connectionpool.py", line 490, in _m

1/1 [==============================] - 0s 19ms/step


Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connection.py", line 196, in _new_conn
    sock = connection.create_connection(
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "C:\Users\ihebm\anaconda3\envs\emotionG7\lib\site-packages\urllib3\connectionpool.py", line 490, in _m

1/1 [==============================] - 0s 18ms/step
